In [2]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import functions as f

In [7]:
stats = {"place": [], "type1": [], "type2": [], "area": []}
df = pd.DataFrame()

for file in os.listdir("./dataset/raw_unprocessed/green_spaces"):
    place = file.split("_")[0]
    type1 = file.split("_")[1]
    type2 = "_".join(file.replace(".parquet", "").replace("_ALL", "").split("_")[2:])

    stats["place"].append(place)
    stats["type1"].append(type1)
    stats["type2"].append(type2)

    x = gpd.read_parquet(f"./dataset/raw_unprocessed/green_spaces/{file}")
    # hey Gino, yoou have to reproject the data before calculating the area
    # dont do this. The files in WGS84 projection
    # x.crs = "EPSG:25832"

    # reproject
    x = x.to_crs(f.DENMARK_CRS)
    x.crs = "EPSG:25832"

    x["place"] = place
    x["type1"] = type1
    x["type2"] = type2
    x["area"] = x[["geometry"]].area/1000000 # square kilometer instead of square meter
    df = pd.concat([df, x])
    

In [8]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df[["type1", "type2", "area"]].groupby(["type1", "type2"]).agg(Total=("area", np.sum), AVG=("area", np.mean), STD=("area", np.std)).reset_index()

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_25024\389643662.py:2: FutureWarning: The provided callable <function sum at 0x000001CBF84794E0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df[["type1", "type2", "area"]].groupby(["type1", "type2"]).agg(Total=("area", np.sum), AVG=("area", np.mean), STD=("area", np.std)).reset_index()
C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_25024\389643662.py:2: FutureWarning: The provided callable <function mean at 0x000001CBF847A5C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df[["type1", "type2", "area"]].groupby(["type1", "type2"]).agg(Total=("area", np.sum), AVG=("area", np.mean), STD=("area", np.std)).reset_index()
C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_25024\389643662.py:

,type1,type2,Total,AVG,STD
0,amenity,grave_yard,7.16022,0.00938,0.01223
1,landuse,allotments,24.65011,0.03840,0.06159
2,landuse,basin,3.44951,0.00115,0.00182
3,landuse,cemetery,13.08256,0.01137,0.02846
4,landuse,farmland,12996.58024,0.14164,0.26354
5,landuse,flowerbed,0.05021,0.00011,0.00081
6,landuse,forest,5339.12448,0.05191,0.43915
7,landuse,grass,101.20598,0.00356,0.01670
8,landuse,greenfield,60.67502,0.04881,0.17237
9,landuse,meadow,1449.10855,0.02829,0.10777


In [ ]:
### Merge areas together 

### Merge areas together

Below code merges forest areas together

In [5]:
gpd_df_forest = gpd.GeoDataFrame()

for file in os.listdir("./dataset/raw_unprocessed/green_spaces"):
    place = file.split("_")[0]
    type1 = file.split("_")[1]
    type2 = "_".join(file.replace(".parquet", "").replace("_ALL", "").split("_")[2:])

    if type1 == "landuse" and type2 == "forest":
        print(place)
        t = gpd.read_parquet(f"./dataset/raw_unprocessed/green_spaces/{file}")
        try:
            display(t.sample(100).explore())
        except:
            pass
        gpd_df_forest = gpd.GeoDataFrame(pd.concat([gpd_df_forest, t], ignore_index=True))
        
    # x = gpd.read_parquet(f"./dataset/raw_unprocessed/green_spaces/{file}")

lolland


midtjylland


nordjylland
syddanmark


zealand


In [16]:
# keep only geometry
gpd_df_forest[["geometry"]].to_parquet("./dataset/raw_unprocessed/denmark_landuse_forest_crs_wgs85.parquet")